In [1]:
# Importar las librerías necesarias
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns  
from pathlib import Path
import glob
import time

from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA  
from sklearn.cluster import KMeans



In [2]:
# Establecer el random seed
random_seed = 42
np.random.seed(random_seed)

# Cargar y Explorar los Datos
Carga el archivo parquet usando pandas y realiza una exploración inicial de los datos, incluyendo estadísticas descriptivas y visualización de distribuciones.

In [3]:
# Cargar todos los archivos parquet que contienen 'merged_inmuebles24_departamentos_duckdb'
data_folder = '../../data/processed/'
pattern = f"{data_folder}*merged_inmuebles24_departamentos_duckdb*.parquet"

In [4]:

# Buscar todos los archivos que coincidan con el patrón
files = glob.glob(pattern)


In [ ]:

print(f"Archivos encontrados: {len(files)}")
for file in files:
    print(f"  - {Path(file).name}")


In [6]:

# Leer y concatenar todos los archivos
if len(files) == 0:
    raise FileNotFoundError(f"No se encontraron archivos con el patrón: {pattern}")


In [ ]:

dfs = []
for file in files:
    temp_df = pd.read_parquet(file)
    temp_df['source_file'] = Path(file).name  # Agregar columna con nombre del archivo fuente
    dfs.append(temp_df)
    print(f"Cargado: {Path(file).name} - {len(temp_df)} registros")


In [8]:
# Concatenar todos los DataFrames
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.shape

### Análisis de duplicados

In [ ]:
# Contar duplicados completos (todas las columnas)
duplicados_completos = df.duplicated().sum()
print(f"\n1. Duplicados completos (todas las columnas): {duplicados_completos}")
print(f"   Porcentaje: {(duplicados_completos/len(df)*100):.2f}%")


In [ ]:
# Identificar columnas clave para duplicados (excluyendo source_file)
cols_clave = [col for col in df.columns if col != 'source_file']
cols_clave


In [ ]:
# Duplicados por columnas clave
duplicados_clave = df.duplicated(subset=cols_clave).sum()
print(f"\n2. Duplicados por columnas clave (excluyendo source_file): {duplicados_clave}")
print(f"   Porcentaje: {(duplicados_clave/len(df)*100):.2f}%")


In [ ]:
# Si hay duplicados, mostrar ejemplos
if duplicados_clave > 0:
    print("\n3. Ejemplos de registros duplicados:")
    mask_duplicados = df.duplicated(subset=cols_clave, keep=False)
    ejemplos_duplicados = df[mask_duplicados].sort_values(by=cols_clave[:3]).head(10)
    display(ejemplos_duplicados[['precio_mxn', 'lote_m2', 'direccion', 'source_file'][:min(5, len(ejemplos_duplicados.columns))]])
    
    # Contar duplicados por archivo fuente
    print("\n4. Distribución de duplicados por archivo fuente:")
    duplicados_por_archivo = df[mask_duplicados].groupby('source_file').size().sort_values(ascending=False)
    print(duplicados_por_archivo)


In [ ]:
# Eliminar duplicados si existen
if duplicados_clave > 0:
    print(f"\n5. Eliminando {duplicados_clave} duplicados...")
    df_original_len = len(df)
    df = df.drop_duplicates(subset=cols_clave, keep='first')
    print(f"   Registros antes: {df_original_len}")
    print(f"   Registros después: {len(df)}")
    print(f"   Registros eliminados: {df_original_len - len(df)}")

### Análisis de Nulos

In [15]:
# Contar nulos por columna
nulos_por_columna = df.isnull().sum().sort_values(ascending=False)
porcentaje_nulos = (nulos_por_columna / len(df) * 100).round(2)

In [16]:
# Crear DataFrame de resumen
resumen_nulos = pd.DataFrame({
    'Valores_Nulos': nulos_por_columna,
    'Porcentaje': porcentaje_nulos,
    'Tipo_Dato': df.dtypes
})

In [ ]:
# Filtrar solo columnas con nulos
resumen_nulos_con_datos = resumen_nulos[resumen_nulos['Valores_Nulos'] > 0]

print(f"\n1. Resumen general:")
print(f"   Total de columnas: {len(df.columns)}")
print(f"   Columnas con nulos: {len(resumen_nulos_con_datos)}")
print(f"   Columnas sin nulos: {len(df.columns) - len(resumen_nulos_con_datos)}")

In [ ]:
# Exploración inicial de los datos
# Mostrar las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
df.head()


In [ ]:
if len(resumen_nulos_con_datos) > 0:
    print(f"\n2. Top 20 columnas con más valores nulos:")
    display(resumen_nulos_con_datos.head(20))
    

In [ ]:
df[df.AGEB.isnull()].head(10)

In [21]:
df = df.dropna(subset=['AGEB'])

In [ ]:
df.shape

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [24]:
df = df.drop_duplicates(subset=['Título'])

In [25]:
df = df.dropna(subset=['latitud', 'longitud'])

In [ ]:
df.shape

In [ ]:
df.isnull().sum().sort_values(ascending=False)[:20]

In [28]:
obj_cols = ['direccion','colonia','cp','neighbourhood','Título','Enlace','colonia_std','municipio',
            'ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'AGEB', 'MZA',
            'key', 'ENTIDAD_fm', 'MUN_fm', 'LOC_fm', 'AGEB_fm',
            'MZA_fm', 'CVEVIAL', 'CVESEG', 'CVEFT', 'NOMVIAL', 'TIPOVIAL',
            'CVEVIAL1', 'CVESEG1', 'CVEREF1', 'TIPOVR1', 'NOMREF1', 'CVEVIAL2',
            'CVESEG2', 'CVEREF2', 'TIPOVR2', 'NOMREF2', 'CVEVIAL3', 'CVESEG3',
            'CVEREF3', 'TIPOVR3', 'NOMREF3', 'municipio_1', 'address', 'road',
            'quarter', 'borough', 'postcode', 'neighbourhood', 'source_file', 
            'lon', 'lat','latitud_1', 'longitud_1','distancia','latitud','longitud']

In [29]:
target = ['precio_mxn']

In [30]:
numeric_cols = list(set(df.columns) - set(obj_cols) - set(target))

In [ ]:
uppercase_columns = [col for col in df[numeric_cols].columns if col.isupper()]
print("Columns with uppercase names:", uppercase_columns)

In [32]:
#porque sabemos de los datos del INEGI que no hay registros.
df[uppercase_columns] = df[uppercase_columns].fillna(0)

In [ ]:
df.isnull().sum().sort_values(ascending=False)[:10]

In [34]:
df['lote_m2'] = df.groupby('colonia')['lote_m2'].transform(lambda x: x.fillna(int(x.mean())))

In [ ]:
df[df['lote_m2'].isnull()].source_file.value_counts()

In [ ]:
# Información general del DataFrame
print("\nInformación general del DataFrame:")
df.info()

In [37]:
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [38]:
fix_int = list(df.select_dtypes('Int64').columns)

In [ ]:
fix_int

In [40]:
df[fix_int] = df[fix_int].astype('int64')

In [ ]:
df.info()

In [ ]:
# Estadísticas descriptivas
print("\nEstadísticas descriptivas:")
df.describe(include='all')

In [ ]:
# Visualización de distribuciones
# Seleccionar columnas numéricas
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Crear subplots para las columnas numéricas
num_cols = len(numeric_columns)
fig, axes = plt.subplots(nrows=(num_cols // 3) + 1, ncols=3, figsize=(15, 5 * ((num_cols // 3) + 1)))
axes = axes.flatten()

for i, column in enumerate(numeric_columns):
    sns.histplot(df[column], kde=True, bins=30, ax=axes[i])
    axes[i].set_title(f"Distribución de {column}")
    axes[i].set_xlabel(column)
    axes[i].set_ylabel("Frecuencia")

# Eliminar ejes vacíos si hay menos subplots que espacios
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
municipio_order = df['municipio'].value_counts().index

sns.countplot(data=df, x='municipio', order=municipio_order, hue='municipio')
plt.xticks(rotation=90)
plt.title('Distribución de municipios')
plt.show()

In [ ]:
df.colonia.value_counts().head(10).plot(kind='bar', figsize=(12,6))
plt.show()

In [ ]:
# Un CP puede contener varias colonias, así que nos quedamos sólo con colonias
df.cp.value_counts().head(10).plot(kind='bar', figsize=(12,6))
plt.show()

In [47]:
top10_cols= list(df.colonia.value_counts().head(10).index)

In [ ]:
df['colonia_top10'] = df['colonia'].where(df['colonia'].isin(top10_cols), 'otros')

# Preprocesamiento de Datos
Aplica técnicas de limpieza de datos, manejo de valores nulos y codificación de variables categóricas.

In [ ]:
# Identificar valores nulos en el DataFrame
print("\nValores nulos por columna:")
df.isnull().sum().sort_values(ascending=False)

In [50]:
# outliers
# Función para detectar outliers usando el método del rango intercuartílico (IQR) 
def detectar_outliers_iqr(data, columna):
    Q1 = data[columna].quantile(0.25)
    Q3 = data[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = data[(data[columna] < limite_inferior) | (data[columna] > limite_superior)]
    return outliers

In [ ]:
# Aplicar la función a las columnas numéricas
for col in numeric_cols:
    outliers = detectar_outliers_iqr(df, col)
    print(f"\nNúmero de outliers en {col}: {len(outliers)}")
    if len(outliers) > 0:
        print(outliers[[col]].head())
    # Visualizar outliers
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot de {col}")  

identificamos 2 outliers evidentes y muy seprados del resto de los datos en lote_m2, los cuales eliminamos

In [ ]:
df.sort_values('lote_m2', ascending=False).head(10)

In [101]:
# Identificar los índices de los dos outliers más grandes
outliers = df.sort_values('lote_m2', ascending=False).head(2).index

In [54]:
# Eliminar los outliers del DataFrame
df = df.drop(outliers)


In [55]:
categorical_columns = list(df.select_dtypes(include=["object", "category"]).columns)
categorical_columns.remove('direccion')

In [56]:
df = pd.get_dummies(df, columns=['municipio','colonia_top10'], drop_first=True)

In [ ]:
df.head()

In [ ]:
df.shape

# Clustering
Realiza un análisis de clustering para agrupar los datos en diferentes segmentos.

In [59]:
df_analysis = df[numeric_cols].copy()

In [60]:
#df_analysis['random'] = np.random.rand(len(df_analysis))

In [61]:
cols = df_analysis.columns

In [62]:
# Escalar los datos
scaler = StandardScaler()
df_analysis[cols] = scaler.fit_transform(df_analysis[cols])

In [63]:
## Arreglar los nulos y quitar esta celda
df_analysis = df_analysis.dropna()

In [ ]:
df_analysis.dtypes[120:]

In [ ]:
# Crear el modelo PCA sin especificar n_components
pca = PCA()
pca.fit(df_analysis)

# Calcular la varianza acumulada
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

#Gráfica de la varianza acumulada
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.title('Varianza acumulada por número de componentes')
plt.xlabel('Número de componentes')
plt.ylabel('Varianza acumulada')
plt.axhline(y=0.95, color='r', linestyle='-')
plt.axvline(x=np.argmax(cumulative_variance >= 0.95) + 1, color='g', linestyle='-')
plt.grid()
plt.show()

# Elegir el número de componentes que expliquen al menos el 95% de la varianza
n_components = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Número óptimo de componentes: {n_components}")



In [ ]:
print("Varianza acumulada:")
print(cumulative_variance)


In [67]:
# Aplicar PCA con el número óptimo de componentes
pca = PCA(n_components=n_components)
pca_data = pca.fit_transform(df_analysis)

In [ ]:
# Importancia de cada componente (varianza explicada)
explained_variance = pca.explained_variance_ratio_
print("Varianza explicada por cada componente:")
for i, var in enumerate(explained_variance):
    print(f"Componente {i+1}: {var:.2%}")

In [ ]:
# Gráfica de varianza explicada por cada componente
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, label='Varianza explicada')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Varianza acumulada')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% de varianza')
plt.xlabel('Número de componentes principales')
plt.ylabel('Proporción de varianza explicada')
plt.title('Varianza explicada por componentes principales')
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
# Contribución de las características originales a los primeros 22 componentes
components = pd.DataFrame(pca.components_[:20], columns=df_analysis.columns)
print("\nContribución de las características originales a los primeros 22 componentes:")
components.T.sort_values(by = 0, ascending=False)

In [ ]:
most_important_features = components.iloc[0].abs().sort_values(ascending=False).head(35)
print("Características más importantes del primer componente:")
print(most_important_features)

In [72]:
# Reconstrucción aproximada de los datos originales
reconstructed_data = pca.inverse_transform(pca_data)

In [ ]:

# Comparar los datos originales con los reconstruidos
print("Datos originales (primeras filas):")
df_analysis.head()

In [ ]:
print("\nDatos reconstruidos (primeras filas):")
pd.DataFrame(reconstructed_data, columns=df_analysis.columns).head()

In [ ]:
# Calcular la inertia para diferentes números de clusters
inertia = []
for k in range(1, 21):  # Probar de 1 a 10 clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pca_data)  # Usa los datos reducidos por PCA
    inertia.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(range(1, 21), inertia, marker='o', linestyle='--')
plt.xlabel('Número de clusters')
plt.ylabel('Inertia')
plt.title('Método del Codo')
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

# Calcular el coeficiente de silueta para diferentes números de clusters
silhouette_scores = []
for k in range(2, 21):  # Probar de 2 a 10 clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pca_data)
    score = silhouette_score(pca_data, kmeans.labels_)
    silhouette_scores.append(score)

# Graficar el coeficiente de silueta
plt.figure(figsize=(8, 5))
plt.plot(range(2, 21), silhouette_scores, marker='o', linestyle='--')
plt.xlabel('Número de clusters')
plt.ylabel('Coeficiente de Silueta')
plt.title('Coeficiente de Silueta para diferentes números de clusters')
plt.grid()
plt.show()

In [103]:
# Aplicar el algoritmo K-Means para clustering
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(df_analysis)

In [ ]:

# Agregar los clusters al DataFrame original
df_analysis['Cluster'] = clusters

# Visualizar los clusters en el espacio reducido por PCA
plt.figure(figsize=(10, 6))
sns.scatterplot(x=pca_data[:, 0], y=pca_data[:, 1], hue=clusters, palette="viridis", s=50)
plt.title("Visualización de Clusters (K-Means)")
plt.xlabel("Componente Principal 1")
plt.ylabel("Componente Principal 2")
plt.legend(title="Cluster")
plt.show()

In [ ]:
import plotly.express as px

# Crear un DataFrame con los datos de PCA y clusters
df_pca = pd.DataFrame(pca_data[:,:3], columns=['PC1', 'PC2', 'PC3'])  # Asegúrate de que pca_data tenga al menos 3 componentes
df_pca['Cluster'] = clusters  # Agregar los clusters al DataFrame

# Graficar en 3D con plotly
fig = px.scatter_3d(
    df_pca,
    x='PC1', y='PC2', z='PC3',  # Ejes del gráfico
    color='Cluster',            # Colorear por cluster
    title="Visualización de Clusters en 3D (K-Means)",
    opacity=0.3,                # Transparencia de los puntos
    size_max=1,                 # Tamaño máximo de los puntos
    symbol_sequence=['circle'], # Usar círculos como símbolo
    color_continuous_scale='Viridis'  # Paleta de colores
)

# Ajustar el tamaño del gráfico
fig.update_layout(
    width=600,  # Ancho del gráfico
    height=600   # Alto del gráfico
)

# Mostrar el gráfico interactivo
fig.show()

In [ ]:
# Mostrar el tamaño de cada cluster
cluster_sizes = df_analysis['Cluster'].value_counts()
print("\nTamaño de cada cluster:")
cluster_sizes

In [ ]:
# Calcular las características promedio de cada cluster
numeric_columns = df_analysis.select_dtypes('number')
cluster_means = numeric_columns.groupby(df_analysis['Cluster']).mean()
cluster_means.T

In [ ]:
df_analysis.columns

In [83]:
loadings = pd.DataFrame(
            pca.components_.T,
            columns=[f'PC{i+1}' for i in range(pca.n_components_)],
            index=df_analysis.drop('Cluster',axis =1).columns
        )

In [84]:
feature_importance_pca = np.sum(
            loadings.abs() * pca.explained_variance_ratio_, axis=1
        )
feature_importance_pca = feature_importance_pca.sort_values(ascending=False)

In [85]:
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
feature_importance_pca.sort_values(ascending=False, inplace = True)

In [ ]:
feature_importance_pca

In [ ]:
feature_importance_pca.index

In [88]:
#random_position = feature_importance_pca.index.get_loc('random')
# Quedarse solo con las features antes de 'random'
#filtered_features = feature_importance_pca.iloc[:random_position]
        

In [89]:
#filtered_features.index

In [90]:
#final_features = list(filtered_features.index)
final_features = list(feature_importance_pca.index)

In [ ]:
# Calcular la matriz de correlación para las características seleccionadas
correlation_matrix = df[final_features].corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(36, 30))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Matriz de Correlación de las Características Seleccionadas")
plt.show()

In [ ]:
# Establecer un umbral para considerar alta correlación
correlation_threshold = 0.60

# Identificar pares de características altamente correlacionadas
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
            high_corr_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))

print(f"Pares de características altamente correlacionadas (umbral > {correlation_threshold}):")
high_corr_pairs

In [ ]:
# Crear un conjunto para almacenar las características a eliminar
features_to_remove = set()

# Para cada par de características correlacionadas, eliminar la menos importante según PCA
for feature1, feature2 in high_corr_pairs:
    if feature_importance_pca[feature1] < feature_importance_pca[feature2]:
        features_to_remove.add(feature1)
    else:
        features_to_remove.add(feature2)

# Filtrar las características finales eliminando las correlacionadas menos importantes
final_features_filtered = [feature for feature in final_features if feature not in features_to_remove]

print(f"Características finales después de eliminar correlacionadas: {len(final_features_filtered)}")
final_features_filtered

In [94]:
final_features = final_features_filtered.copy()

# Entrenamiento de Modelos

## Regresión Lineal

In [ ]:
# Regresión Lineal con Pipeline Completo
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import time

# Seleccionar las características (X) y la variable objetivo (y)
target_column = 'precio_mxn'

# Filtrar features antes de 'random' si existe
if 'random' in final_features:
    final_features_filtered = final_features[:final_features.index('random')]
    print(f"Filtradas {len(final_features_filtered)} features antes de 'random'")
else:
    final_features_filtered = final_features
    print(f"Usando todas las {len(final_features_filtered)} features")

X = df[final_features_filtered]
y = df[target_column]

print(f"Forma del dataset: X {X.shape}, y {y.shape}")
print(f"Features seleccionadas: {final_features_filtered}")

# Verificar valores faltantes
print(f"\nValores faltantes en X: {X.isnull().sum().sum()}")
print(f"Valores faltantes en y: {y.isnull().sum()}")

# Eliminar filas con valores faltantes si los hay
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    print(f"Después de eliminar NaN: X {X.shape}, y {y.shape}")

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=None  # Para regresión no usamos stratify
)

print(f"\nDivisión de datos:")
print(f"Entrenamiento: X {X_train.shape}, y {y_train.shape}")
print(f"Prueba: X {X_test.shape}, y {y_test.shape}")

# Crear el pipeline con StandardScaler y LinearRegression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Entrenar el modelo
print("\nEntrenando el modelo...")
start_time = time.time()
pipeline.fit(X_train, y_train)
end_time = time.time()

training_time = end_time - start_time 
print(f"Entrenamiento completado en {training_time:.2f} segundos.")

# Realizar predicciones
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

# Evaluar el modelo
def evaluate_model(y_true, y_pred, dataset_name=""):
    """Función para evaluar el modelo con múltiples métricas"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\nResultados {dataset_name}:")
    print(f"Error cuadrático medio (MSE): {mse:,.2f}")
    print(f"Raíz del error cuadrático medio (RMSE): {rmse:,.2f}")
    print(f"Error absoluto medio (MAE): {mae:,.2f}")
    print(f"Coeficiente de determinación (R²): {r2:.4f}")
    
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

# Evaluar en entrenamiento y prueba
train_metrics = evaluate_model(y_train, y_pred_train, "Entrenamiento")
test_metrics = evaluate_model(y_test, y_pred_test, "Prueba")

# Validación cruzada
print(f"\nValidación cruzada (5-fold):")
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
print(f"R² promedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

cv_scores_rmse = cross_val_score(pipeline, X_train, y_train, cv=5, 
                                scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores_rmse)
print(f"RMSE promedio: {cv_rmse.mean():,.2f} (+/- {cv_rmse.std() * 2:,.2f})")

# Análisis de coeficientes
scaler = pipeline.named_steps['scaler']
regressor = pipeline.named_steps['regressor']

# Obtener coeficientes e importancia
feature_importance = pd.DataFrame({
    'feature': final_features_filtered,
    'coefficient': regressor.coef_,
    'abs_coefficient': np.abs(regressor.coef_)
}).sort_values('abs_coefficient', ascending=False)

print(f"\nTop 10 características más importantes:")
print(feature_importance.head(10).to_string(index=False))

# Visualizaciones mejoradas
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Predicciones vs Valores Reales (Prueba)
axes[0,0].scatter(y_test, y_pred_test, alpha=0.6, color='blue', s=30)
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
               color='red', linestyle='--', linewidth=2)
axes[0,0].set_title(f"Predicciones vs Valores Reales (Prueba)\nR² = {test_metrics['r2']:.4f}")
axes[0,0].set_xlabel("Valores Reales")
axes[0,0].set_ylabel("Predicciones")
axes[0,0].grid(True, alpha=0.3)

# 2. Residuos vs Predicciones
residuals_test = y_test - y_pred_test
axes[0,1].scatter(y_pred_test, residuals_test, alpha=0.6, color='green', s=30)
axes[0,1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0,1].set_title("Residuos vs Predicciones")
axes[0,1].set_xlabel("Predicciones")
axes[0,1].set_ylabel("Residuos")
axes[0,1].grid(True, alpha=0.3)

# 3. Distribución de residuos
axes[1,0].hist(residuals_test, bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[1,0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1,0].set_title("Distribución de Residuos")
axes[1,0].set_xlabel("Residuos")
axes[1,0].set_ylabel("Frecuencia")
axes[1,0].grid(True, alpha=0.3)

# 4. Importancia de características (Top 10)
top_10_features = feature_importance.head(10)
axes[1,1].barh(range(len(top_10_features)), top_10_features['abs_coefficient'])
axes[1,1].set_yticks(range(len(top_10_features)))
axes[1,1].set_yticklabels(top_10_features['feature'])
axes[1,1].set_title("Top 10 Características Más Importantes")
axes[1,1].set_xlabel("Valor Absoluto del Coeficiente")
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Comparación Entrenamiento vs Prueba
comparison_df = pd.DataFrame({
    'Métrica': ['MSE', 'RMSE', 'MAE', 'R²'],
    'Entrenamiento': [train_metrics['mse'], train_metrics['rmse'], 
                     train_metrics['mae'], train_metrics['r2']],
    'Prueba': [test_metrics['mse'], test_metrics['rmse'], 
              test_metrics['mae'], test_metrics['r2']]
})

print(f"\nComparación Entrenamiento vs Prueba:")
print(comparison_df.to_string(index=False))

# Detectar posible overfitting
if train_metrics['r2'] - test_metrics['r2'] > 0.1:
    print(f"\n⚠️  Posible overfitting detectado:")
    print(f"   Diferencia R²: {train_metrics['r2'] - test_metrics['r2']:.4f}")
elif test_metrics['r2'] < 0.5:
    print(f"\n⚠️  Modelo con bajo rendimiento:")
    print(f"   R² en prueba: {test_metrics['r2']:.4f}")
else:
    print(f"\n✅ Modelo con buen rendimiento y generalización")

# Guardar el modelo entrenado y métricas
model_results_linear = {
    'pipeline': pipeline,
    'train_metrics': train_metrics,
    'test_metrics': test_metrics,
    'feature_importance': feature_importance,
    'cv_scores': cv_scores,
    'training_time': training_time
}

print(f"\nModelo entrenado exitosamente con {len(final_features_filtered)} características")


### RandomForestRegressor

In [ ]:
# Regresión No Lineal con Pipeline (Random Forest Regressor)
from sklearn.model_selection import train_test_split, cross_val_score
# Importamos el nuevo modelo: Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --- Asunciones de variables (DEBEN estar definidas en el entorno global) ---
# df, final_features, y target_column deben estar disponibles antes de la ejecución
# Ejemplo de variables si no estuvieran definidas:
# data_size = 500
# final_features = [f'feature_{i}' for i in range(5)]
# df = pd.DataFrame(np.random.rand(data_size, 5), columns=final_features)
# df['precio_mxn'] = df.iloc[:, 0] * 100 + df.iloc[:, 1] * 50 + np.random.randn(data_size) * 10
# target_column = 'precio_mxn'
# -------------------------------------------------------------------------

# Seleccionar las características (X) y la variable objetivo (y)
# Suponiendo que df, final_features y target_column ya están cargados/definidos
# Aquí usaremos un pequeño ejemplo para que el código sea runnable de forma autónoma:
try:
    X = df[final_features_filtered]
    y = df[target_column]
except NameError:
    print("⚠️ Usando datos de ejemplo ya que 'df' no está definido en el entorno.")
    data_size = 1000
    final_features = [f'area_m2', 'num_habitaciones', 'distancia_centro_km', 'antiguedad_anos', 'dummy_col']
    df = pd.DataFrame(np.random.rand(data_size, 5), columns=final_features)
    df['precio_mxn'] = (df['area_m2'] * 500000) + (df['num_habitaciones'] * 100000) + (np.random.randn(data_size) * 50000)
    target_column = 'precio_mxn'
    final_features_filtered = final_features
    X = df[final_features_filtered]
    y = df[target_column]


print(f"Forma del dataset: X {X.shape}, y {y.shape}")
print(f"Features seleccionadas: {final_features_filtered}")

# Verificar y limpiar valores faltantes (mismo código que el original)
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    print(f"Después de eliminar NaN: X {X.shape}, y {y.shape}")

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"\nDivisión de datos:")
print(f"Entrenamiento: X {X_train.shape}, y {y_train.shape}")
print(f"Prueba: X {X_test.shape}, y {y_test.shape}")

# --- CAMBIO CLAVE: Reemplazar LinearRegression por RandomForestRegressor ---
# Random Forest es un modelo de ensamble basado en árboles que captura no linealidades.
# El StandardScaler (escalamiento) es menos crítico aquí, pero se mantiene en el pipeline.
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', RandomForestRegressor(
        n_estimators=100,  # Número de árboles en el bosque
        max_depth=10,      # Profundidad máxima de los árboles para evitar sobreajuste
        random_state=42,
        n_jobs=-1          # Usa todos los núcleos de CPU disponibles
    ))
])

# Entrenar el modelo
print("\nEntrenando el modelo Random Forest...")
start_time = time.time()
pipeline.fit(X_train, y_train)
end_time = time.time()

training_time = end_time - start_time
print(f"Entrenamiento completado en {training_time:.2f} segundos.")

# Realizar predicciones
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

# Evaluar el modelo (función de evaluación reutilizada)
def evaluate_model(y_true, y_pred, dataset_name=""):
    """Función para evaluar el modelo con múltiples métricas"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\nResultados {dataset_name}:")
    print(f"Error cuadrático medio (MSE): {mse:,.2f}")
    print(f"Raíz del error cuadrático medio (RMSE): {rmse:,.2f}")
    print(f"Error absoluto medio (MAE): {mae:,.2f}")
    print(f"Coeficiente de determinación (R²): {r2:.4f}")
    
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

# Evaluar en entrenamiento y prueba
train_metrics = evaluate_model(y_train, y_pred_train, "Entrenamiento (Random Forest)")
test_metrics = evaluate_model(y_test, y_pred_test, "Prueba (Random Forest)")

# Validación cruzada
print(f"\nValidación cruzada (5-fold):")
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
print(f"R² promedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

cv_scores_rmse = cross_val_score(pipeline, X_train, y_train, cv=5, 
                                scoring='neg_mean_squared_error', n_jobs=-1)
cv_rmse = np.sqrt(-cv_scores_rmse)
print(f"RMSE promedio: {cv_rmse.mean():,.2f} (+/- {cv_rmse.std() * 2:,.2f})")

# Análisis de Importancia de Características (diferente al de Regresión Lineal)
regressor = pipeline.named_steps['regressor']

# Obtener la importancia de las características del Random Forest
feature_importance = pd.DataFrame({
    'feature': final_features_filtered,
    'importance': regressor.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 características más importantes (Random Forest):")
# Usamos 'importance' en lugar de 'abs_coefficient'
print(feature_importance.head(10).to_string(index=False))

# Visualizaciones mejoradas (reutilizando la estructura de la gráfica)
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Predicciones vs Valores Reales (Prueba)
axes[0,0].scatter(y_test, y_pred_test, alpha=0.6, color='#FF5733', s=30)
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
               color='#33FF57', linestyle='--', linewidth=2)
axes[0,0].set_title(f"Predicciones vs Valores Reales (Prueba)\nR² = {test_metrics['r2']:.4f}", 
                    fontsize=14, color='#333333')
axes[0,0].set_xlabel("Valores Reales (precio_mxn)", fontsize=12)
axes[0,0].set_ylabel("Predicciones (precio_mxn)", fontsize=12)
axes[0,0].grid(True, alpha=0.3, linestyle=':')

# 2. Residuos vs Predicciones
residuals_test = y_test - y_pred_test
axes[0,1].scatter(y_pred_test, residuals_test, alpha=0.6, color='#33AFFF', s=30)
axes[0,1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0,1].set_title("Residuos vs Predicciones (Idealmente aleatorio)", 
                    fontsize=14, color='#333333')
axes[0,1].set_xlabel("Predicciones", fontsize=12)
axes[0,1].set_ylabel("Residuos (Error)", fontsize=12)
axes[0,1].grid(True, alpha=0.3, linestyle=':')

# 3. Distribución de residuos
axes[1,0].hist(residuals_test, bins=30, alpha=0.8, color='#AFFF33', edgecolor='black')
axes[1,0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1,0].set_title("Distribución de Residuos (Idealmente normal y centrado en 0)", 
                    fontsize=14, color='#333333')
axes[1,0].set_xlabel("Residuos", fontsize=12)
axes[1,0].set_ylabel("Frecuencia", fontsize=12)
axes[1,0].grid(True, alpha=0.3, linestyle=':')

# 4. Importancia de características (Top 10)
top_10_features = feature_importance.head(10)
axes[1,1].barh(range(len(top_10_features)), top_10_features['importance'], color='#9A33FF')
axes[1,1].set_yticks(range(len(top_10_features)))
axes[1,1].set_yticklabels(top_10_features['feature'], fontsize=10)
axes[1,1].set_title("Top 10 Características Más Importantes (Random Forest)", 
                    fontsize=14, color='#333333')
axes[1,1].set_xlabel("Importancia (Gini Impurity)", fontsize=12)
axes[1,1].invert_yaxis() # Pone la más importante arriba
axes[1,1].grid(axis='x', alpha=0.3, linestyle=':')

plt.suptitle("Evaluación del Modelo Random Forest Regressor", fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

# Comparación Entrenamiento vs Prueba
comparison_df = pd.DataFrame({
    'Métrica': ['MSE', 'RMSE', 'MAE', 'R²'],
    'Entrenamiento': [train_metrics['mse'], train_metrics['rmse'], 
                     train_metrics['mae'], train_metrics['r2']],
    'Prueba': [test_metrics['mse'], test_metrics['rmse'], 
              test_metrics['mae'], test_metrics['r2']]
})

print(f"\nComparación Entrenamiento vs Prueba:")
print(comparison_df.to_string(index=False))

# Detectar posible overfitting
# En Random Forest, la diferencia R² tiende a ser mayor, pero el R² de prueba debe ser alto.
r2_diff = train_metrics['r2'] - test_metrics['r2']
if r2_diff > 0.15: # Un umbral más alto para modelos de árbol
    print(f"\n⚠️  Random Forest detectado con posible sobreajuste (Overfitting):")
    print(f"   Diferencia R² (Entrenamiento - Prueba): {r2_diff:.4f}")
    print("   Considera reducir max_depth o aumentar min_samples_split.")
elif test_metrics['r2'] < 0.6:
    print(f"\n⚠️  Modelo con bajo rendimiento:")
    print(f"   R² en prueba: {test_metrics['r2']:.4f}")
else:
    print(f"\n✅ Modelo Random Forest con buen rendimiento y generalización.")

# Guardar el modelo entrenado y métricas (actualizado)
model_results_rf = {
    'pipeline': pipeline,
    'train_metrics': train_metrics,
    'test_metrics': test_metrics,
    'feature_importance': feature_importance,
    'cv_scores': cv_scores,
    'training_time': training_time
}

print(f"\nModelo Random Forest entrenado exitosamente con {len(final_features_filtered)} características")


## GradientBoostingRegressor

In [ ]:
# Regresión No Lineal con Pipeline (Gradient Boosting Regressor)
from sklearn.model_selection import train_test_split, cross_val_score
# Importamos el nuevo modelo: Gradient Boosting Regressor (GBM)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --- Asunciones de variables (DEBEN estar definidas en el entorno global) ---
# df, final_features, y target_column deben estar disponibles antes de la ejecución
# Ejemplo de variables si no estuvieran definidas:
# data_size = 500
# final_features = [f'feature_{i}' for i in range(5)]
# df = pd.DataFrame(np.random.rand(data_size, 5), columns=final_features)
# df['precio_mxn'] = df.iloc[:, 0] * 100 + df.iloc[:, 1] * 50 + np.random.randn(data_size) * 10
# target_column = 'precio_mxn'
# -------------------------------------------------------------------------

# Seleccionar las características (X) y la variable objetivo (y)
# Suponiendo que df, final_features y target_column ya están cargados/definidos
# Aquí usaremos un pequeño ejemplo para que el código sea runnable de forma autónoma:
try:
    X = df[final_features_filtered]
    y = df[target_column]
except NameError:
    print("⚠️ Usando datos de ejemplo ya que 'df' no está definido en el entorno.")
    data_size = 1000
    final_features = [f'area_m2', 'num_habitaciones', 'distancia_centro_km', 'antiguedad_anos', 'dummy_col']
    df = pd.DataFrame(np.random.rand(data_size, 5), columns=final_features)
    df['precio_mxn'] = (df['area_m2'] * 500000) + (df['num_habitaciones'] * 100000) + (np.random.randn(data_size) * 50000)
    target_column = 'precio_mxn'
    final_features_filtered = final_features
    X = df[final_features_filtered]
    y = df[target_column]


print(f"Forma del dataset: X {X.shape}, y {y.shape}")
print(f"Features seleccionadas: {final_features_filtered}")

# Verificar y limpiar valores faltantes (mismo código que el original)
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    print(f"Después de eliminar NaN: X {X.shape}, y {y.shape}")

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"\nDivisión de datos:")
print(f"Entrenamiento: X {X_train.shape}, y {y_train.shape}")
print(f"Prueba: X {X_test.shape}, y {y_test.shape}")

# --- CAMBIO CLAVE: Reemplazar RandomForestRegressor por GradientBoostingRegressor (GBM) ---
# GBM es un modelo de ensamble basado en boosting (aprendizaje secuencial) que a menudo 
# ofrece un mejor rendimiento que Random Forest.
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', GradientBoostingRegressor(
        n_estimators=100,      # Número de etapas de boosting (árboles)
        learning_rate=0.1,     # Tasa de aprendizaje, controla la contribución de cada árbol
        max_depth=3,           # Profundidad máxima de cada árbol (se prefieren árboles poco profundos)
        random_state=42
    ))
])

# Entrenar el modelo
print("\nEntrenando el modelo Gradient Boosting Regressor (GBM)...")
start_time = time.time()
pipeline.fit(X_train, y_train)
end_time = time.time()

training_time = end_time - start_time
print(f"Entrenamiento completado en {training_time:.2f} segundos.")

# Realizar predicciones
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

# Evaluar el modelo (función de evaluación reutilizada)
def evaluate_model(y_true, y_pred, dataset_name=""):
    """Función para evaluar el modelo con múltiples métricas"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\nResultados {dataset_name}:")
    print(f"Error cuadrático medio (MSE): {mse:,.2f}")
    print(f"Raíz del error cuadrático medio (RMSE): {rmse:,.2f}")
    print(f"Error absoluto medio (MAE): {mae:,.2f}")
    print(f"Coeficiente de determinación (R²): {r2:.4f}")
    
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

# Evaluar en entrenamiento y prueba
train_metrics = evaluate_model(y_train, y_pred_train, "Entrenamiento (GBM)")
test_metrics = evaluate_model(y_test, y_pred_test, "Prueba (GBM)")

# Validación cruzada
print(f"\nValidación cruzada (5-fold):")
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
print(f"R² promedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

cv_scores_rmse = cross_val_score(pipeline, X_train, y_train, cv=5, 
                                scoring='neg_mean_squared_error', n_jobs=-1)
cv_rmse = np.sqrt(-cv_scores_rmse)
print(f"RMSE promedio: {cv_rmse.mean():,.2f} (+/- {cv_rmse.std() * 2:,.2f})")

# Análisis de Importancia de Características (misma forma que Random Forest)
regressor = pipeline.named_steps['regressor']

# Obtener la importancia de las características del Gradient Boosting
feature_importance = pd.DataFrame({
    'feature': final_features_filtered,
    'importance': regressor.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 características más importantes (Gradient Boosting):")
print(feature_importance.head(10).to_string(index=False))

# Visualizaciones mejoradas (reutilizando la estructura de la gráfica)
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Predicciones vs Valores Reales (Prueba)
axes[0,0].scatter(y_test, y_pred_test, alpha=0.6, color='#FF5733', s=30)
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
               color='#33FF57', linestyle='--', linewidth=2)
axes[0,0].set_title(f"Predicciones vs Valores Reales (Prueba - GBM)\nR² = {test_metrics['r2']:.4f}", 
                    fontsize=14, color='#333333')
axes[0,0].set_xlabel("Valores Reales (precio_mxn)", fontsize=12)
axes[0,0].set_ylabel("Predicciones (precio_mxn)", fontsize=12)
axes[0,0].grid(True, alpha=0.3, linestyle=':')

# 2. Residuos vs Predicciones
residuals_test = y_test - y_pred_test
axes[0,1].scatter(y_pred_test, residuals_test, alpha=0.6, color='#33AFFF', s=30)
axes[0,1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0,1].set_title("Residuos vs Predicciones (GBM)", 
                    fontsize=14, color='#333333')
axes[0,1].set_xlabel("Predicciones", fontsize=12)
axes[0,1].set_ylabel("Residuos (Error)", fontsize=12)
axes[0,1].grid(True, alpha=0.3, linestyle=':')

# 3. Distribución de residuos
axes[1,0].hist(residuals_test, bins=30, alpha=0.8, color='#AFFF33', edgecolor='black')
axes[1,0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1,0].set_title("Distribución de Residuos (GBM)", 
                    fontsize=14, color='#333333')
axes[1,0].set_xlabel("Residuos", fontsize=12)
axes[1,0].set_ylabel("Frecuencia", fontsize=12)
axes[1,0].grid(True, alpha=0.3, linestyle=':')

# 4. Importancia de características (Top 10)
top_10_features = feature_importance.head(10)
axes[1,1].barh(range(len(top_10_features)), top_10_features['importance'], color='#9A33FF')
axes[1,1].set_yticks(range(len(top_10_features)))
axes[1,1].set_yticklabels(top_10_features['feature'], fontsize=10)
axes[1,1].set_title("Top 10 Características Más Importantes (GBM)", 
                    fontsize=14, color='#333333')
axes[1,1].set_xlabel("Importancia", fontsize=12)
axes[1,1].invert_yaxis() # Pone la más importante arriba
axes[1,1].grid(axis='x', alpha=0.3, linestyle=':')

plt.suptitle("Evaluación del Modelo Gradient Boosting Regressor (GBM)", fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

# Comparación Entrenamiento vs Prueba
comparison_df = pd.DataFrame({
    'Métrica': ['MSE', 'RMSE', 'MAE', 'R²'],
    'Entrenamiento': [train_metrics['mse'], train_metrics['rmse'], 
                     train_metrics['mae'], train_metrics['r2']],
    'Prueba': [test_metrics['mse'], test_metrics['rmse'], 
              test_metrics['mae'], test_metrics['r2']]
})

print(f"\nComparación Entrenamiento vs Prueba:")
print(comparison_df.to_string(index=False))

# Detectar posible overfitting
# GBM es más propenso al sobreajuste que Random Forest, por lo que el umbral es más estricto.
r2_diff = train_metrics['r2'] - test_metrics['r2']
if r2_diff > 0.08: # Bajamos el umbral para modelos de boosting (0.08 es más estricto)
    print(f"\n⚠️  Gradient Boosting detectado con posible sobreajuste (Overfitting):")
    print(f"   Diferencia R² (Entrenamiento - Prueba): {r2_diff:.4f}")
    print("   Considera reducir el 'learning_rate', aumentar 'n_estimators' o usar 'subsample'.")
elif test_metrics['r2'] < 0.7:
    print(f"\n⚠️  Modelo con bajo rendimiento:")
    print(f"   R² en prueba: {test_metrics['r2']:.4f}")
else:
    print(f"\n✅ Modelo Gradient Boosting con buen rendimiento y generalización.")

# Guardar el modelo entrenado y métricas (actualizado)
model_results_gbm = {
    'pipeline': pipeline,
    'train_metrics': train_metrics,
    'test_metrics': test_metrics,
    'feature_importance': feature_importance,
    'cv_scores': cv_scores,
    'training_time': training_time
}

print(f"\nModelo Gradient Boosting entrenado exitosamente con {len(final_features_filtered)} características")


In [ ]:
# Regresión No Lineal Avanzada con Pipeline (XGBoost Regressor)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# --- Importamos el modelo de la librería externa: XGBoost Regressor ---
try:
    from xgboost import XGBRegressor
except ImportError:
    # Mensaje de error si la librería XGBoost no está instalada (solo para entorno real)
    print("Error: La librería 'xgboost' no está instalada. Por favor, instálela con 'pip install xgboost'")
    raise

# --- Asunciones de variables (DEBEN estar definidas en el entorno global) ---
# df, final_features, y target_column deben estar disponibles antes de la ejecución
# Aquí usamos un pequeño ejemplo para que el código sea runnable de forma autónoma:
try:
    X = df[final_features_filtered]
    y = df[target_column]
except NameError:
    print("⚠️ Usando datos de ejemplo ya que 'df' no está definido en el entorno.")
    data_size = 1000
    final_features = [f'area_m2', 'num_habitaciones', 'distancia_centro_km', 'antiguedad_anos', 'dummy_col']
    df = pd.DataFrame(np.random.rand(data_size, 5), columns=final_features)
    df['precio_mxn'] = (df['area_m2'] * 500000) + (df['num_habitaciones'] * 100000) + (np.random.randn(data_size) * 50000)
    target_column = 'precio_mxn'
    final_features_filtered = final_features
    X = df[final_features_filtered]
    y = df[target_column]


print(f"Forma del dataset: X {X.shape}, y {y.shape}")
print(f"Features seleccionadas: {final_features_filtered}")

# Verificar y limpiar valores faltantes (mismo código que el original)
if X.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    mask = ~(X.isnull().any(axis=1) | y.isnull())
    X = X[mask]
    y = y[mask]
    print(f"Después de eliminar NaN: X {X.shape}, y {y.shape}")

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"\nDivisión de datos:")
print(f"Entrenamiento: X {X_train.shape}, y {y_train.shape}")
print(f"Prueba: X {X_test.shape}, y {y_test.shape}")

# --- CAMBIO CLAVE: Reemplazar GradientBoostingRegressor por XGBRegressor ---
# XGBoost es el modelo de Boosting optimizado y regularizado.
pipeline = Pipeline([
    # XGBoost no requiere escalado, pero lo mantenemos para consistencia
    ('scaler', StandardScaler()), 
    ('regressor', XGBRegressor(
        n_estimators=100,      # Número de árboles (iters)
        learning_rate=0.1,     # Tasa de aprendizaje
        max_depth=5,           # Profundidad máxima (un poco más profunda que GBM)
        random_state=42,
        n_jobs=-1,             # Utiliza todos los núcleos de la CPU para rapidez
        # Parámetros de regularización por defecto (l1 y l2)
        objective='reg:squarederror' # Objetivo para regresión
    ))
])

# Entrenar el modelo
print("\nEntrenando el modelo XGBoost Regressor...")
start_time = time.time()
pipeline.fit(X_train, y_train)
end_time = time.time()

training_time = end_time - start_time
print(f"Entrenamiento completado en {training_time:.2f} segundos.")

# Realizar predicciones
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

# Evaluar el modelo (función de evaluación reutilizada)
def evaluate_model(y_true, y_pred, dataset_name=""):
    """Función para evaluar el modelo con múltiples métricas"""
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\nResultados {dataset_name}:")
    print(f"Error cuadrático medio (MSE): {mse:,.2f}")
    print(f"Raíz del error cuadrático medio (RMSE): {rmse:,.2f}")
    print(f"Error absoluto medio (MAE): {mae:,.2f}")
    print(f"Coeficiente de determinación (R²): {r2:.4f}")
    
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

# Evaluar en entrenamiento y prueba
train_metrics = evaluate_model(y_train, y_pred_train, "Entrenamiento (XGBoost)")
test_metrics = evaluate_model(y_test, y_pred_test, "Prueba (XGBoost)")

# Validación cruzada
print(f"\nValidación cruzada (5-fold):")
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2', n_jobs=-1)
print(f"R² promedio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

cv_scores_rmse = cross_val_score(pipeline, X_train, y_train, cv=5, 
                                scoring='neg_mean_squared_error', n_jobs=-1)
cv_rmse = np.sqrt(-cv_scores_rmse)
print(f"RMSE promedio: {cv_rmse.mean():,.2f} (+/- {cv_rmse.std() * 2:,.2f})")

# Análisis de Importancia de Características
regressor = pipeline.named_steps['regressor']

# Obtener la importancia de las características de XGBoost
feature_importance = pd.DataFrame({
    'feature': final_features_filtered,
    'importance': regressor.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nTop 10 características más importantes (XGBoost):")
print(feature_importance.head(10).to_string(index=False))

# Visualizaciones mejoradas
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Predicciones vs Valores Reales (Prueba)
axes[0,0].scatter(y_test, y_pred_test, alpha=0.6, color='#00796B', s=30) # Nuevo color
axes[0,0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
               color='#FFAB00', linestyle='--', linewidth=2)
axes[0,0].set_title(f"Predicciones vs Valores Reales (Prueba - XGBoost)\nR² = {test_metrics['r2']:.4f}", 
                    fontsize=14, color='#333333')
axes[0,0].set_xlabel("Valores Reales (precio_mxn)", fontsize=12)
axes[0,0].set_ylabel("Predicciones (precio_mxn)", fontsize=12)
axes[0,0].grid(True, alpha=0.3, linestyle=':')

# 2. Residuos vs Predicciones
residuals_test = y_test - y_pred_test
axes[0,1].scatter(y_pred_test, residuals_test, alpha=0.6, color='#5D4037', s=30)
axes[0,1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0,1].set_title("Residuos vs Predicciones (XGBoost)", 
                    fontsize=14, color='#333333')
axes[0,1].set_xlabel("Predicciones", fontsize=12)
axes[0,1].set_ylabel("Residuos (Error)", fontsize=12)
axes[0,1].grid(True, alpha=0.3, linestyle=':')

# 3. Distribución de residuos
axes[1,0].hist(residuals_test, bins=30, alpha=0.8, color='#1E88E5', edgecolor='black')
axes[1,0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1,0].set_title("Distribución de Residuos (XGBoost)", 
                    fontsize=14, color='#333333')
axes[1,0].set_xlabel("Residuos", fontsize=12)
axes[1,0].set_ylabel("Frecuencia", fontsize=12)
axes[1,0].grid(True, alpha=0.3, linestyle=':')

# 4. Importancia de características (Top 10)
top_10_features = feature_importance.head(10)
axes[1,1].barh(range(len(top_10_features)), top_10_features['importance'], color='#FF7043')
axes[1,1].set_yticks(range(len(top_10_features)))
axes[1,1].set_yticklabels(top_10_features['feature'], fontsize=10)
axes[1,1].set_title("Top 10 Características Más Importantes (XGBoost)", 
                    fontsize=14, color='#333333')
axes[1,1].set_xlabel("Importancia", fontsize=12)
axes[1,1].invert_yaxis() # Pone la más importante arriba
axes[1,1].grid(axis='x', alpha=0.3, linestyle=':')

plt.suptitle("Evaluación del Modelo XGBoost Regressor", fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

# Comparación Entrenamiento vs Prueba
comparison_df = pd.DataFrame({
    'Métrica': ['MSE', 'RMSE', 'MAE', 'R²'],
    'Entrenamiento': [train_metrics['mse'], train_metrics['rmse'], 
                     train_metrics['mae'], train_metrics['r2']],
    'Prueba': [test_metrics['mse'], test_metrics['rmse'], 
              test_metrics['mae'], test_metrics['r2']]
})

print(f"\nComparación Entrenamiento vs Prueba:")
print(comparison_df.to_string(index=False))

# Detectar posible overfitting
r2_diff = train_metrics['r2'] - test_metrics['r2']
if r2_diff > 0.05: # Umbral más estricto para XGBoost
    print(f"\n⚠️  XGBoost detectado con posible sobreajuste (Overfitting):")
    print(f"   Diferencia R² (Entrenamiento - Prueba): {r2_diff:.4f}")
    print("   Considera aumentar el valor de 'reg_alpha' (L1) o 'reg_lambda' (L2), o reducir 'max_depth'.")
elif test_metrics['r2'] < 0.75:
    print(f"\n⚠️  Modelo con bajo rendimiento para ser XGBoost:")
    print(f"   R² en prueba: {test_metrics['r2']:.4f}")
else:
    print(f"\n✅ Modelo XGBoost con muy buen rendimiento y generalización.")

# Guardar el modelo entrenado y métricas (actualizado)
model_results_xgb = {
    'pipeline': pipeline,
    'train_metrics': train_metrics,
    'test_metrics': test_metrics,
    'feature_importance': feature_importance,
    'cv_scores': cv_scores,
    'training_time': training_time
}

print(f"\nModelo XGBoost entrenado exitosamente con {len(final_features_filtered)} características")


In [ ]:
# Crear un diccionario para almacenar los resultados de los modelos
model_comparison = {}

# Lista de modelos entrenados previamente
models = {
    "Linear Regression": model_results_linear,  # Supongamos que este es el resultado del modelo de regresión lineal
    "Random Forest": model_results_rf,          # Resultado del modelo Random Forest
    "Gradient Boosting": model_results_gbm,     # Resultado del modelo Gradient Boosting
    "XGBoost": model_results_xgb                # Resultado del modelo XGBoost
}

# Evaluar cada modelo
for model_name, model_result in models.items():
    pipeline = model_result['pipeline']
    test_metrics = model_result['test_metrics']
    
    # Medir el tiempo de predicción
    start_time = time.time()
    y_pred_test = pipeline.predict(X_test)
    prediction_time = time.time() - start_time
    
    # Guardar los resultados en el diccionario
    model_comparison[model_name] = {
        "R² (Prueba)": test_metrics['r2'],
        "RMSE (Prueba)": test_metrics['rmse'],
        "MAE (Prueba)": test_metrics['mae'],
        "Tiempo de predicción (s)": prediction_time,
        "Tiempo de entrenamiento (s)": model_result.get('training_time', 'N/A'),  # Si se almacenó el tiempo de entrenamiento
        "Interpretabilidad": "Alta" if model_name == "Linear Regression" else "Media" if model_name == "Random Forest" else "Baja"
    }

# Crear un DataFrame para visualizar la comparación
comparison_df = pd.DataFrame(model_comparison).T

# Ordenar por R² en prueba
comparison_df = comparison_df.sort_values(by="R² (Prueba)", ascending=False)

print("Comparación de Modelos:")
print(comparison_df)

In [ ]:
# Decidir el modelo final
best_model_name = comparison_df.index[0]
print(f"\nEl modelo seleccionado para producción es: {best_model_name}")